In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
dataset_name = 'narySt/text_detoxification_dataset'
model_name = 'ToirovSadi/t5-small'

In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset(dataset_name)
dataset = dataset['train'].train_test_split(test_size=10_000)
dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'reference', 'similarity', 'lenght_diff', 'trn_tox', 'ref_tox', 'tox_diff', 'tox_diff_abs', 'translation_words', 'reference_words', '__index_level_0__'],
        num_rows: 567777
    })
    test: Dataset({
        features: ['translation', 'reference', 'similarity', 'lenght_diff', 'trn_tox', 'ref_tox', 'tox_diff', 'tox_diff_abs', 'translation_words', 'reference_words', '__index_level_0__'],
        num_rows: 10000
    })
})

In [5]:
def preprocess_data(sample):
    toxic_sent = sample['translation']
    neutral_sent = sample['reference']
    
    toxic_val = sample['trn_tox']
    neutral_val = sample['ref_tox']
    
    if toxic_val < neutral_val: # swap
        toxic_sent, neutral_sent = neutral_sent, toxic_sent
        
    return {
        'toxic_sent': toxic_sent,
        'neutral_sent': neutral_sent,
    }

dataset = dataset.map(preprocess_data, batched=False)

Map:   0%|          | 0/567777 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Preprocessing data

In [6]:
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [8]:
tokenizer('hey how are you?', max_length=2, truncation=True, padding=True)

{'input_ids': [3, 1], 'attention_mask': [1, 1]}

In [9]:
prefix = "paraphrase: "
max_length = 128

def preprocess_function(samples):
    toxic_sent = [prefix + sent for sent in samples['toxic_sent']]
    neutral_sent = samples['neutral_sent']
    
    model_inputs = tokenizer(toxic_sent, max_length=max_length, truncation=True)
    labels = tokenizer(neutral_sent, max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/567777 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'reference', 'similarity', 'lenght_diff', 'trn_tox', 'ref_tox', 'tox_diff', 'tox_diff_abs', 'translation_words', 'reference_words', '__index_level_0__', 'toxic_sent', 'neutral_sent', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 567777
    })
    test: Dataset({
        features: ['translation', 'reference', 'similarity', 'lenght_diff', 'trn_tox', 'ref_tox', 'tox_diff', 'tox_diff_abs', 'translation_words', 'reference_words', '__index_level_0__', 'toxic_sent', 'neutral_sent', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
})

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [13]:
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

import evaluate
import numpy as np

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Replace -100 in the predictions as we can't decode them.
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    b_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value * 100 for key, value in result.items()}
    result["bleu"] = b_result["bleu"]
    
    return {k: round(v, 4) for k, v in result.items()}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Train model

In [14]:
batch_size = 8

training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small",
    evaluation_strategy="steps",
    eval_steps=70000,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    report_to="tensorboard",
    push_to_hub=True,
    generation_max_length=max_length,
    generation_num_beams=3,
)

In [15]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu
70000,1.743300,1.600013,60.980200,37.687200,59.156100,59.164400,0.282100
140000,1.721600,1.590479,61.124100,37.872200,59.287800,59.299100,0.283900
210000,1.738900,1.587134,61.140400,37.885400,59.295300,59.308100,0.284300


TrainOutput(global_step=212919, training_loss=1.7294391208917586, metrics={'train_runtime': 13121.6392, 'train_samples_per_second': 129.811, 'train_steps_per_second': 16.227, 'total_flos': 1.6425295405940736e+16, 'train_loss': 1.7294391208917586, 'epoch': 3.0})

In [17]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1703932132.nsjmhjr540.242.0:   0%|          | 0.00/75.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ToirovSadi/t5-small/commit/f1fd66825ac3a8bd434a451eb6f7a79df45a3032', commit_message='End of training', commit_description='', oid='f1fd66825ac3a8bd434a451eb6f7a79df45a3032', pr_url=None, pr_revision=None, pr_num=None)